In [1]:
import pandas as pd
import pymongo as pm
import elasticsearch

In [2]:
import sys
sys.path.append('..')
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
import lib.obo
import lib.classification.elastic
import lib.classification.validation
import lib.utils
import data
import pymonad

reload(lib.classification.validation)
reload(lib.classification.elastic)
reload(lib.obo)
reload(lib.utils)
reload(data)

from lib.classification.elastic import annotate_index, collapse_matches, build_synonyms_graph, search_term
from lib.utils import map_series, expand_dataframe, expand, flatten
from lib.classification.validation import correct_ratio, incorrect_ratio, perfect_match
from geo_annotation.import_ontology_es import import_ontology
from elasticsearch import Elasticsearch
from lib.obo import read_ontology, Ontology
pd.set_option('display.width', 512)

In [3]:
es = elasticsearch.Elasticsearch()

In [4]:
normal_terms = ['normal', 'control', 'reference', 'healthy']

In [5]:
norms = search_term(es=es,
            term=normal_terms,
            index='samples',
            fields=['title', 'characteristics_raw', 'source_name'])

In [6]:
len(norms)

315535

In [8]:
norms_all = search_term(es=es,
            term=normal_terms,
            fields=['title', 'characteristics_raw', 'source_name', 'description'],
            index='samples')
len(norms_all)

409931

In [9]:
norms_df = pd.DataFrame(dict(accession=list(map('GSM{}'.format, norms))))
norms_df.head()

,accession
0,GSM305072
1,GSM305305
2,GSM305310
3,GSM1422841
4,GSM1422846


In [11]:
norms_all_df = pd.DataFrame(dict(accession=list(map('GSM{}'.format, norms_all))))
norms_all_df.head()

,accession
0,GSM1063144
1,GSM855974
2,GSM305072
3,GSM305305
4,GSM305310


In [13]:
norms_df.to_pickle('../data/geo-annotation/norms.py.pickle')

In [12]:
norms_all_df.to_pickle('../data/geo-annotation/norms_all.py.pickle')

In [15]:
db = pm.MongoClient().scraper_meta

In [16]:
samples = pd.DataFrame(list(db.samples.find({}, {'_id': 0, 'accession': 1, 'series': 1})))
samples.head()

,accession,series
0,GSM1,[GSE506]
1,GSM2,[GSE506]
2,GSM3,[GSE462]
3,GSM4,[GSE462]
4,GSM5,[GSE462]


In [17]:
samples_series = pd.DataFrame.from_records([(accession, serie) 
                                      for _, (accession, series) in samples.iterrows() 
                                      for serie in series],
                                    columns=['accession', 'series'])
samples_series.head()

,accession,series
0,GSM1,GSE506
1,GSM2,GSE506
2,GSM3,GSE462
3,GSM4,GSE462
4,GSM5,GSE462


In [20]:
norms_df['norm'] = 1
norms_df.head()

,accession,norm
0,GSM305072,1
1,GSM305305,1
2,GSM305310,1
3,GSM1422841,1
4,GSM1422846,1


In [19]:
norms_all_df['norm'] = 1
norms_all_df.head()

,accession,norm
0,GSM1063144,1
1,GSM855974,1
2,GSM305072,1
3,GSM305305,1
4,GSM305310,1


In [33]:
samples_with_norms = (
    pd.merge(norms_df, samples_series, how='right')
    .drop_duplicates(subset=['accession', 'series'])
    .fillna(0)
)
print(samples_with_norms.shape)
samples_with_norms.head()

(1730022, 3)


,accession,norm,series
0,GSM305072,1,GSE12103
4,GSM305305,1,GSE12103
8,GSM305310,1,GSE12103
12,GSM1422841,1,GSE58949
13,GSM1422846,1,GSE58949


## Norms count in dataset analysis

In [36]:
(
    samples_with_norms
    .groupby('series')
    .agg(dict(norm=lambda norms: sum(norms)/len(norms), accession=len))
    .query('norm > 0')
#     .accession
#     .sum()
    .shape
)

(19646, 2)

In [41]:
(
    samples_with_norms
    .groupby('series')
    .agg(dict(norm=lambda norms: sum(norms)/len(norms), accession=len))
    .query('norm > 0.5')
#     .accession
#     .sum()
    .head()
#     .shape
)

,accession,norm
series,,
GSE10004,3,1.000000
GSE10005,48,1.000000
GSE10008,38,0.578947
GSE10010,6,1.000000
GSE10025,137,1.000000


In [42]:
samples_with_norms.query('series == "GSE10005"')

,accession,norm,series
142136,GSM252280,1,GSE10005
142138,GSM252295,1,GSE10005
142140,GSM252301,1,GSE10005
142142,GSM252309,1,GSE10005
142144,GSM252316,1,GSE10005
142146,GSM252320,1,GSE10005
142148,GSM252323,1,GSE10005
142150,GSM252325,1,GSE10005
142152,GSM252332,1,GSE10005
142154,GSM252337,1,GSE10005


In [44]:
samples_with_norms.to_pickle('../data/geo-annotation/samples.norms.res2.pickle')

In [3]:
samples_with_norms = pd.read_pickle('../data/geo-annotation/samples.norms.res2.pickle')

In [5]:
samples_with_norms.query('series == "GSE10024"')

,accession,norm,series
570289,GSM253288,1,GSE10024
571112,GSM253287,1,GSE10024
852154,GSM253283,0,GSE10024
852155,GSM253284,0,GSE10024
852156,GSM253285,0,GSE10024
852157,GSM253286,0,GSE10024
